**About** : This notebook is used to prepare the data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

### Imports

In [ ]:
import os
import sys
import glob
import json
import h5py
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from copy import deepcopy
from joblib import delayed
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from params import *
from util.plots import *

from data.preparation import prepare_data, prepare_xenocanto_data
from data.dataset import WaveDataset
from data.transforms import get_transfos

from util.logger import create_logger, save_config, prepare_log_folder
from util.torch import init_distributed

from model_zoo.models import define_model
from model_zoo.melspec import FeatureExtractor, CustomTimeMasking, CustomFreqMasking

from training.main import k_fold

### Data

In [ ]:
df = prepare_data()
df.sample(5)

In [ ]:
transforms = get_transfos(augment=True, strength=0)

dataset = WaveDataset(
    df,
    transforms,
    max_len=32000 * 5,
    train=True
)

In [ ]:
df.iloc[479]

In [ ]:
# for i in tqdm(range(len(dataset))):
#     # try:
#     x = dataset[i]
#     # except:
#     #     print(i)
#     #     break

In [ ]:
wave, y, w = dataset[np.random.choice(len(dataset))]

In [ ]:
display_audio(wave.numpy(), title=", ".join(([CLASSES[int(c)] for c in np.argwhere(y.numpy())[0]])))

### Spectrograms

In [ ]:
params = {
    "sample_rate": 32000,
    "n_mels": 128,
    "f_min": 20,
    "n_fft": 2048,
    "hop_length": 512,
    "normalized": True,
}

spec_augment_config = {
    "freq_mask": {
        "mask_max_length": 10,
        "mask_max_masks": 3,
        "p": 0.3,
    },
    "time_mask": {
        "mask_max_length": 20,
        "mask_max_masks": 3,
        "p": 0.3,
    },
}

In [ ]:
ft_extractor = FeatureExtractor(params, spec_augment_config=spec_augment_config)

In [ ]:
melspec = ft_extractor(wave.unsqueeze(0))[0]

plot_spectrogram(melspec.numpy(), params)

### Model

In [ ]:
model = define_model(
    "tf_efficientnetv2_s", 
    params,
    head="freq_att",
    n_channels=1,
    num_classes=NUM_CLASSES
)

In [ ]:
y = model(wave.unsqueeze(0))
y.size()

### Train

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    duration = 5
    aug_strength = 1
    use_secondary_labels = True
    normalize = True

    melspec_config = {
        "sample_rate": 32000,
        "n_mels": 128,
        "f_min": 20,
        "n_fft": 2048,
        "hop_length": 512,
        "normalized": True,
    }

    mixup_config = {
        "p": 0,
        "additive": True,
        "alpha": 4
    }

    spec_augment_config = {
        "freq_mask": {
            "mask_max_length": 10,
            "mask_max_masks": 3,
            "p": 0.3,
        },
        "time_mask": {
            "mask_max_length": 20,
            "mask_max_masks": 3,
            "p": 0.3,
        },
    }

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
    name = "tf_efficientnetv2_s"  # convnextv2_tiny maxvit_tiny_tf_384
    pretrained_weights = None

    num_classes = 182
    drop_rate = 0.1
    drop_path_rate = 0.1
    n_channels = 1
    head = "gem"
    increase_stride = False

    # Training
    loss_config = {
        "name": "bce",
        "weighted": False,
        "smoothing": 0.0,
        "activation": "sigmoid",
    }

    data_config = {
        "batch_size": 32,
        "val_bs": 32,
        "mix": "cutmix",
        "num_classes": num_classes,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "Ranger",
        "lr": 5e-4,
        "warmup_prop": 0.0,
        "betas": (0.9, 0.999),
        "max_grad_norm": 0.1,
        "weight_decay": 0.0,
    }

    epochs = 1

    use_fp16 = True
    verbose = 1
    verbose_eval = 100

    fullfit = False
    n_fullfit = 1

In [ ]:
DEBUG = True

In [ ]:
log_folder = None
config = Config

if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'\n -> Logging results to {log_folder}\n')

    create_logger(directory=log_folder, name="logs.txt")
    save_config(config, log_folder + "config.json")

init_distributed(config)

In [ ]:
df = prepare_data(DATA_PATH)
# df_xc = prepare_xenocanto_data(DATA_PATH)
# df = pd.concat([df, df_xc], ignore_index=True)

# if DEBUG:
#     df = df.sample(100).reset_index(drop=True)

In [ ]:
k_fold(config, df, log_folder=log_folder)

Done !